# Search Algorithm : Sudoku

---

This notebook covers basic search algorithms for **Solving Sudoku Grid**.       

You will see *uninformed search* algorithms, namely **Breadth First Search**, **Depth First Search**, **Uniform Cost Search**, **Depth Limited Search** and **Iterative Depth Search**. The setup is a typical Sudoku puzzle, utilizing a 9x9 grid. In total, there are 81 cells in the puzzle, with 9 cells contained in each of the 9 blocks. Some cells will have numbers that are pre-filled. To solve the puzzle, the agent must fill up all 81 cells with a number from 1 to 9 inclusive, where each row and column of the grid contains each number exactly once. In this Jupyter Notebook, you will learn how to create and display a sudoku grid clearly, and then apply the various search algorithms on the grid to solve the puzzle.



---

### Essential Libraries

Let us begin by importing the essential Python Libraries.

> Deque : Double-ended queue to perform insertion and deletion operations efficiently from both ends of the queue. Used primarily in BFS    

> Heapq : To implement heaps and the functions associated with them to in initialize priority queue. Used primarily in UCS and A* Search 

> Queue : Library to work with FIFO, LIFO, Priority queues

> Time: Module to measure the execution time of each algorithm

> Pygame: Module to create better looking games

>Sys:  Module that provides access to some variables used or maintained by the Python interpreter, as well as to functions that interact strongly with the interpreter


In [174]:
from collections import deque
import queue
import heapq
import time
import pygame
import sys

---
## Create the Sudoku puzzle as a 2D Grid

Let us create the sudoku puzzle as a standard two-dimensional grid, with unfilled (blank) and filled (hints) positions.

In [175]:
#Function to create sudoku grid that will be solved using the various search algorithms
def display_sudoku(grid):
    print("+-------+-------+-------+")
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("+-------+-------+-------+")
        for j in range(9):
            if j % 3 == 0:
                print("|", end=" ")
            if grid[i][j] == 0:
                print(".", end=" ")
            else:
                print(grid[i][j], end=" ")
            if j == 8:
                print("|")
    print("+-------+-------+-------+")

# Sudoku grid to be displayed
grid = [[0,0,7,2,8,0,0,0,0], 
        [0,0,0,0,0,0,5,0,6],
        [4,1,3,0,0,6,0,8,0],
        [7,2,0,3,9,0,0,0,0],
        [3,4,0,0,0,0,8,1,0],
        [6,8,0,1,0,7,0,0,2],
        [0,0,0,6,7,4,0,2,3],
        [0,0,0,0,0,5,7,0,0],
        [1,0,6,0,2,3,0,4,0]]


display_sudoku(grid)


+-------+-------+-------+
| . . 7 | 2 8 . | . . . |
| . . . | . . . | 5 . 6 |
| 4 1 3 | . . 6 | . 8 . |
+-------+-------+-------+
| 7 2 . | 3 9 . | . . . |
| 3 4 . | . . . | 8 1 . |
| 6 8 . | 1 . 7 | . . 2 |
+-------+-------+-------+
| . . . | 6 7 4 | . 2 3 |
| . . . | . . 5 | 7 . . |
| 1 . 6 | . 2 3 | . 4 . |
+-------+-------+-------+


---
## Helper functions

The following are functions used in all of the aforementioned search algorithms.

In [176]:
# Function to check if the grid is solved
def is_solved(grid):
    for row in range(9):
        for col in range(9):
            if grid[row][col] == 0:
                return False
    return True

# Function to find an empty cell in the grid
def find_empty_cell(grid):
    for row in range(9):
        for col in range(9):
            if grid[row][col] == 0:
                return (row, col)
    return None

# Function to check if a number can be placed in a given position
def is_valid_move(grid, row, col, num):
    # Check row
    if num in grid[row]:
        return False

    # Check column
    for i in range(9):
        if grid[i][col] == num:
            return False

    # Check 3x3 square
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if grid[start_row + i][start_col + j] == num:
                return False

    return True

# Function to measure the time taken by an algorithm
def measure_time(algorithm, grid, num_trials=100000):
    total_time = 0
    for _ in range(num_trials):
        start_time = time.perf_counter()
        solution = algorithm(grid)
        end_time = time.perf_counter()
        total_time += end_time - start_time
    average_time = total_time / num_trials
    return average_time, solution


---
## Solution codes using each Search Algorithm
---

### Breadth-First-Search (BFS)

**Uninformed Search** : Focuses on exploring the Sudoku grid structure systematically, without considering the cost from the start state or the distance from the goal state.

In [184]:
def solve_sudoku_bfs(grid):
    # Create a deque to use as a queue
    queue = deque()
    # Add the initial grid to the queue
    queue.append(grid)

    # Continue processing until the queue is empty
    while queue:
        # Take the first grid from the queue
        current_grid = queue.popleft()

        # Check if the current grid is solved
        if is_solved(current_grid):
            # If it's solved, return the solved grid
            return current_grid

        # Find an empty cell in the current grid
        empty_cell = find_empty_cell(current_grid)
        row, col = empty_cell

        # Try placing numbers from 1 to 9 in the empty cell
        for num in range(1, 10):
            # Check if placing 'num' in the empty cell is a valid move
            if is_valid_move(current_grid, row, col, num):
                # If it's a valid move, create a new grid with 'num' placed in the empty cell
                new_grid = [row[:] for row in current_grid]
                new_grid[row][col] = num
                # Add the new grid to the queue to explore further
                queue.append(new_grid)

    # If no solution is found, return None
    return None

# Sample Sudoku grid
grid_to_solve1 = grid

# Solve Sudoku using BFS
solved_grid1 = solve_sudoku_bfs(grid_to_solve1)

# Display the solved Sudoku grid
display_sudoku(solved_grid1)


+-------+-------+-------+
| 5 6 7 | 2 8 9 | 4 3 1 |
| 8 9 2 | 4 3 1 | 5 7 6 |
| 4 1 3 | 7 5 6 | 2 8 9 |
+-------+-------+-------+
| 7 2 1 | 3 9 8 | 6 5 4 |
| 3 4 9 | 5 6 2 | 8 1 7 |
| 6 8 5 | 1 4 7 | 3 9 2 |
+-------+-------+-------+
| 9 5 8 | 6 7 4 | 1 2 3 |
| 2 3 4 | 9 1 5 | 7 6 8 |
| 1 7 6 | 8 2 3 | 9 4 5 |
+-------+-------+-------+


### Depth-First-Search (DFS)

**Uninformed Search** : Focuses on exploring the Sudoku grid structure systematically, without considering the cost from the start state or the distance from the goal state.

In [185]:
def solve_sudoku_dfs(grid):
    # Find an empty cell in the grid
    empty_cell = find_empty_cell(grid)
    # If there are no empty cells left, the sudoku is solved
    if not empty_cell:
        return grid

    # Extract row and column from the empty cell
    row, col = empty_cell

    # Try placing numbers from 1 to 9 in the empty cell
    for num in range(1, 10):
        # Check if placing 'num' in the empty cell is a valid move
        if is_valid_move(grid, row, col, num):
            # If it's a valid move, update the grid with 'num' at the empty cell
            grid[row][col] = num
            # Recursively call solve_sudoku_dfs with the updated grid
            # If the recursion returns a valid solution, return the solved grid
            if solve_sudoku_dfs(grid):
                return grid
            # If the current placement of 'num' doesn't lead to a solution, backtrack
            grid[row][col] = 0

    # If no solution is found, return None
    return None


# Sample Sudoku grid
grid_to_solve2 = grid

# Solve Sudoku using DFS
solved_grid2 = solve_sudoku_dfs(grid_to_solve2)

# Display the solved Sudoku grid
display_sudoku(solved_grid2)


+-------+-------+-------+
| 5 6 7 | 2 8 9 | 4 3 1 |
| 8 9 2 | 4 3 1 | 5 7 6 |
| 4 1 3 | 7 5 6 | 2 8 9 |
+-------+-------+-------+
| 7 2 1 | 3 9 8 | 6 5 4 |
| 3 4 9 | 5 6 2 | 8 1 7 |
| 6 8 5 | 1 4 7 | 3 9 2 |
+-------+-------+-------+
| 9 5 8 | 6 7 4 | 1 2 3 |
| 2 3 4 | 9 1 5 | 7 6 8 |
| 1 7 6 | 8 2 3 | 9 4 5 |
+-------+-------+-------+


### Uniform-Cost-Search (UCS)

**Uninformed Search** : Focuses on the Sudoku grid and the cost from start, but not the distance from goal.

In [186]:
import heapq

def solve_sudoku_ucs(grid):
    # Initialize a priority queue (heap) with a tuple containing the cost and the grid
    heap = [(0, grid)]
    # Heapify the list to maintain the heap property
    heapq.heapify(heap)
    # Initialize a set to keep track of visited states
    visited = set()

    # Continue searching until the heap is empty
    while heap:
        # Pop the grid with the lowest cost from the heap
        cost, current_grid = heapq.heappop(heap)
        # Check if the current grid is solved
        if is_solved(current_grid):
            # If it's solved, return the solved grid
            return current_grid

        # Find an empty cell in the current grid
        empty_cell = find_empty_cell(current_grid)
        row, col = empty_cell

        # Try placing numbers from 1 to 9 in the empty cell
        for num in range(1, 10):
            # Check if placing 'num' in the empty cell is a valid move
            if is_valid_move(current_grid, row, col, num):
                # Create a new grid with 'num' placed in the empty cell
                new_grid = [row[:] for row in current_grid]
                new_grid[row][col] = num
                # Calculate the cost of the new grid
                new_cost = calculate_cost(new_grid)
                # Convert the grid to a tuple of tuples for hashability
                grid_tuple = tuple(map(tuple, new_grid))
                # Check if the new grid has not been visited before
                if grid_tuple not in visited:
                    # Add the new grid to the heap with its cost
                    heapq.heappush(heap, (new_cost, new_grid))
                    # Mark the new grid as visited
                    visited.add(grid_tuple)

    # If no solution is found, return None
    return None

# Cost and conflict functions used to evaluate cost, which is the cumulative number of conflicts in the grid
def calculate_cost(grid):
    cost = 0
    # Iterate through each cell in the grid
    for row in range(9):
        for col in range(9):
            # If the cell is empty, increment the cost
            if grid[row][col] == 0:
                cost += 1
            else:
                # Otherwise, count the conflicts for the cell and add to the cost
                cost += count_conflicts(grid, row, col)
    return cost

# Function to count conflicts for a given cell in the grid
def count_conflicts(grid, row, col):
    conflicts = 0
    num = grid[row][col]
    # Check for conflicts in the same row
    for i in range(9):
        if grid[row][i] == num and i != col:
            conflicts += 1
    # Check for conflicts in the same column
    for i in range(9):
        if grid[i][col] == num and i != row:
            conflicts += 1
    # Check for conflicts in the same 3x3 subgrid
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(3):
        for j in range(3):
            if grid[start_row + i][start_col + j] == num and (start_row + i != row or start_col + j != col):
                conflicts += 1
    return conflicts

# Sample Sudoku grid
grid_to_solve3 = grid

# Solve Sudoku using UCS
solved_grid3 = solve_sudoku_ucs(grid_to_solve3)

# Display the solved Sudoku grid
display_sudoku(solved_grid3)



+-------+-------+-------+
| 5 6 7 | 2 8 9 | 4 3 1 |
| 8 9 2 | 4 3 1 | 5 7 6 |
| 4 1 3 | 7 5 6 | 2 8 9 |
+-------+-------+-------+
| 7 2 1 | 3 9 8 | 6 5 4 |
| 3 4 9 | 5 6 2 | 8 1 7 |
| 6 8 5 | 1 4 7 | 3 9 2 |
+-------+-------+-------+
| 9 5 8 | 6 7 4 | 1 2 3 |
| 2 3 4 | 9 1 5 | 7 6 8 |
| 1 7 6 | 8 2 3 | 9 4 5 |
+-------+-------+-------+


### Depth-Limited-Search (DLS)

**Uninformed Search** :  Focuses on the graph structure and the depth from the start, but not the distance from the goal.

In [187]:
def solve_sudoku_dls(grid):
    # Find all empty cells in the grid
    empty_cells = [(row, col) for row in range(9) for col in range(9) if grid[row][col] == 0]
    # Call the depth-limited search function with the grid and empty cells
    return dls(grid, empty_cells)

def dls(grid, empty_cells, depth=0):
    # If the depth reaches the total number of empty cells, the sudoku is solved
    if depth == len(empty_cells):
        return grid

    # Extract the row and column of the current empty cell
    row, col = empty_cells[depth]

    # Try placing numbers from 1 to 9 in the current empty cell
    for num in range(1, 10):
        # Check if placing 'num' in the empty cell is a valid move
        if is_valid_move(grid, row, col, num):
            # If it's a valid move, update the grid with 'num' at the empty cell
            grid[row][col] = num
            # Recursively call dls with the updated grid and the next empty cell
            result = dls(grid, empty_cells, depth + 1)
            # If the recursion returns a valid solution, return the solved grid
            if result:
                return result
            # If the current placement of 'num' doesn't lead to a solution, backtrack
            grid[row][col] = 0

    # If no solution is found, return None
    return None

# Sample Sudoku grid
grid_to_solve4 = grid

# Solve Sudoku using DLS
solved_grid4 = solve_sudoku_dls(grid_to_solve4)

# Display the solved Sudoku grid
display_sudoku(solved_grid4)


+-------+-------+-------+
| 5 6 7 | 2 8 9 | 4 3 1 |
| 8 9 2 | 4 3 1 | 5 7 6 |
| 4 1 3 | 7 5 6 | 2 8 9 |
+-------+-------+-------+
| 7 2 1 | 3 9 8 | 6 5 4 |
| 3 4 9 | 5 6 2 | 8 1 7 |
| 6 8 5 | 1 4 7 | 3 9 2 |
+-------+-------+-------+
| 9 5 8 | 6 7 4 | 1 2 3 |
| 2 3 4 | 9 1 5 | 7 6 8 |
| 1 7 6 | 8 2 3 | 9 4 5 |
+-------+-------+-------+


### Iterative-Depth-Search (IDS)

**Uninformed Search** :  Focuses on the graph structure and the cost from the start, but not the distance from the goal.

In [189]:
def solve_sudoku_ids(grid):
    # Iterate over depths from 1 to 81 (maximum possible depth in Sudoku)
    for depth in range(1, 82):
        # Call depth-limited search with the current depth
        result = dls(grid, depth)
        # If a solution is found, return it
        if result:
            return result
    # If no solution is found within the depth limit, return None
    return None

def dls(grid, depth):
    # Find all empty cells in the grid
    empty_cells = [(row, col) for row in range(9) for col in range(9) if grid[row][col] == 0]
    # Call the recursive depth-limited search function with the grid, empty cells, and depth
    return dls_recursive(grid, empty_cells, depth)

def dls_recursive(grid, empty_cells, depth):
    # If depth reaches 0, return None
    if depth == 0:
        return None

    # If there are no more empty cells, the Sudoku is solved
    if not empty_cells:
        return grid

    # Extract the row and column of the current empty cell
    row, col = empty_cells[0]

    # Try placing numbers from 1 to 9 in the current empty cell
    for num in range(1, 10):
        # Check if placing 'num' in the empty cell is a valid move
        if is_valid_move(grid, row, col, num):
            # If it's a valid move, update the grid with 'num' at the empty cell
            grid[row][col] = num
            # Recursively call the function with the updated grid, remaining empty cells, and reduced depth
            result = dls_recursive(grid, empty_cells[1:], depth - 1)
            # If the recursion returns a valid solution, return the solved grid
            if result:
                return result
            # If the current placement of 'num' doesn't lead to a solution, backtrack
            grid[row][col] = 0

    # If no solution is found, return None
    return None

# Sample Sudoku grid
grid_to_solve5 = grid

# Solve Sudoku using IDS
solved_grid5 = solve_sudoku_ids(grid_to_solve5)

# Display the solved Sudoku grid
display_sudoku(solved_grid5)


+-------+-------+-------+
| 5 6 7 | 2 8 9 | 4 3 1 |
| 8 9 2 | 4 3 1 | 5 7 6 |
| 4 1 3 | 7 5 6 | 2 8 9 |
+-------+-------+-------+
| 7 2 1 | 3 9 8 | 6 5 4 |
| 3 4 9 | 5 6 2 | 8 1 7 |
| 6 8 5 | 1 4 7 | 3 9 2 |
+-------+-------+-------+
| 9 5 8 | 6 7 4 | 1 2 3 |
| 2 3 4 | 9 1 5 | 7 6 8 |
| 1 7 6 | 8 2 3 | 9 4 5 |
+-------+-------+-------+


---
## Time taken for each Search Algorithm

We now measure the time taken for each search algorithm to run. For improved accuracy, we take the average time of 100000 trials for each search algorithm. The ideology behind this is to use runtime as an evaluator for performace of the search algorithms.

In [191]:
#Time taken for BFS
average_time_bfs, solution = measure_time(solve_sudoku_bfs, grid)
print("Average time taken by BFS algorithm (100000 trials):", average_time_bfs)

average_time_dfs, solution = measure_time(solve_sudoku_dfs, grid)
print("\nAverage time taken by DFS algorithm (100000 trials):", average_time_dfs)

average_time_ucs, solution = measure_time(solve_sudoku_ucs, grid)
print("\nAverage time taken by UCS algorithm (100000 trials):", average_time_ucs)

average_time_dls, solution = measure_time(solve_sudoku_dls, grid)
print("\nAverage time taken by DLS algorithm (100000 trials):", average_time_dls)

average_time_ids, solution = measure_time(solve_sudoku_ids, grid)
print("\nAverage time taken by IDS algorithm (100000 trials):", average_time_ids)

Average time taken by BFS algorithm (100000 trials): 1.1336892995750531e-05

Average time taken by DFS algorithm (100000 trials): 1.0024612001398055e-05

Average time taken by UCS algorithm (100000 trials): 1.301106300095853e-05

Average time taken by DLS algorithm (100000 trials): 2.2775542997442245e-05

Average time taken by IDS algorithm (100000 trials): 1.2388744004510954e-05


### Analysis of Algorithms

Upon running the cell multiple times, we can safely conclude that generally speaking, all 5 search algorithms used have similar runtimes. Thereby, they are all similar in performance, in using runtime as the metric.

---
## Create the Sudoku puzzle using Pygame

Let us also create the same sudoku grid being used throughout this notebook using Pygame. This is to demonstrate a more "game-like" aesthetic for the puzzle.

In [192]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Set the dimensions of the Sudoku grid
GRID_WIDTH = 450
GRID_HEIGHT = 450
CELL_SIZE = GRID_WIDTH // 9

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GREY = (200, 200, 200)

# Define the Sudoku grid
grid = [[0, 0, 7, 2, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 5, 0, 6],
        [4, 1, 3, 0, 0, 6, 0, 8, 0],
        [7, 2, 0, 3, 9, 0, 0, 0, 0],
        [3, 4, 0, 0, 0, 0, 8, 1, 0],
        [6, 8, 0, 1, 0, 7, 0, 0, 2],
        [0, 0, 0, 6, 7, 4, 0, 2, 3],
        [0, 0, 0, 0, 0, 5, 7, 0, 0],
        [1, 0, 6, 0, 2, 3, 0, 4, 0]]

# Function to draw the Sudoku grid
def draw_grid(screen):
    screen.fill(WHITE)
    for i in range(10):
        if i % 3 == 0:
            pygame.draw.line(screen, BLACK, (i * CELL_SIZE, 0), (i * CELL_SIZE, GRID_HEIGHT), 4)
            pygame.draw.line(screen, BLACK, (0, i * CELL_SIZE), (GRID_WIDTH, i * CELL_SIZE), 4)
        else:
            pygame.draw.line(screen, BLACK, (i * CELL_SIZE, 0), (i * CELL_SIZE, GRID_HEIGHT), 2)
            pygame.draw.line(screen, BLACK, (0, i * CELL_SIZE), (GRID_WIDTH, i * CELL_SIZE), 2)

# Function to draw numbers on the Sudoku grid
def draw_numbers(screen, grid):
    font = pygame.font.SysFont('Arial', 30)
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                text = font.render(str(grid[i][j]), True, BLACK)
                text_rect = text.get_rect(center=(j * CELL_SIZE + CELL_SIZE // 2, i * CELL_SIZE + CELL_SIZE // 2))
                screen.blit(text, text_rect)

# Main function to create the Sudoku grid
def main():
    screen = pygame.display.set_mode((GRID_WIDTH, GRID_HEIGHT))
    pygame.display.set_caption('Sudoku Grid')
    clock = pygame.time.Clock()
    
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        draw_grid(screen)
        draw_numbers(screen, grid)
        
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()
    sys.exit()

if __name__ == '__main__':
    main()


SystemExit: 